Hate Speech Filtering

Using Tf-Idf and BoW to build SVM model

Linear model? Or RBF model?

RACIST texts classified with '1'
NON-RACIST texts classified with '0'


In [1]:
# Libraries

# Web scraper libraries
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import csv

# NLTK
import re 
from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# ML libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
# import matplotlib.pyplot as plt
logReg = LogisticRegression()



In [2]:
def getLink(pageDriver, baseUrl):
    # Get all 'a-tags'
    a_tags = driver.find_elements_by_tag_name('a')

    # Get all urls on page
    urls = [tag.get_attribute('href') for tag in a_tags]

    #Only base url and all related urls
    useful_urls = [url for url in urls if url and baseUrl in url]

    return useful_urls

def getText(pageDriver):
    # Get textual content from 'body' html tag
    pageText = pageDriver.find_element_by_xpath("/html/body")
    if not pageText:
        return ""
    # pageText is webElement, pageText.text returns texts
    return pageText.text


visited_urls = set()
unvisited_urls = set()

options = webdriver.ChromeOptions()
# run Chrome tab without interface
options.add_argument("--headless")
driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)

baseUrl = input("Please enter url of website: ")
# Add base url to unvisited
unvisited_urls.add(baseUrl)

# counter for text indexing
ct = 31963
while unvisited_urls:
    # Pop url from unvisited url set and add it to visited
    page = unvisited_urls.pop()
    visited_urls.add(page)

    # Get page content
    driver.get(page)

    # Get links
    links = getLink(driver, baseUrl)

    # Get text
    text = getText(driver)
    textToWrite = text.splitlines()
    # Create .csv file
    with open(f'testCSV.csv', 'w') as csvfile:
        fieldnames = ['id', 'text']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        # Write to .csv file with index and text
        for text in textToWrite:
            writer.writerow({'id': ct, 'text': {text}})
            ct += 1

print("WEB SCRAPING FINISHED")



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/Users/tonycui/.wdm/drivers/chromedriver/mac64_m1/100.0.4896.60/chromedriver] found in cache
/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_11884/1559815854.py:28: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(), options=options)
/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_11884/1559815854.py:3: DeprecationWarning: find_elements_by_tag_name is deprecated. Please use find_elements(by=By.TAG_NAME, value=name) instead
  a_tags = driver.find_elements_by_tag_name('a')


WEB SCRAPING FINISHED


/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_11884/1559815854.py:15: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  pageText = pageDriver.find_element_by_xpath("/html/body")


In [3]:
# Read raw .csv data using Pandas Dataframe
train = pd.read_csv('train.csv')
test = pd.read_csv('testCSV.csv')



TF-IDF (term frequency-inverse document frequency) is a statistical measure that evaluates how relevant a word is to a document in a collection of documents.
This is done by multiplying two metrics: how many times a word appears in a document, and the inverse document frequency of the word across a set of documents.

Ref: 
    https://monkeylearn.com/blog/what-is-tf-idf/#:~:text=TF%2DIDF%20(term%20frequency%2D,across%20a%20set%20of%20documents


Doc:
    https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

Bag of words is a Natural Language Processing technique of text modelling. 
In technical terms, we can say that it is a method of feature extraction with text data.

Ref: 
    https://www.mygreatlearning.com/blog/bag-of-words/


Doc:
    https://scikit-learn.org/stable/modules/feature_extraction.html

Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.

Ref/Doc:
    https://scikit-learn.org/stable/modules/svm.html#:~:text=Support%20vector%20machines%20(SVMs)%20are,than%20the%20number%20of%20samples

In [4]:
# Combine training data with test data
allTexts = train.append(test, ignore_index=True, sort=False)

# allTexts.tail()

/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_11884/4265950438.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  allTexts = train.append(test, ignore_index=True, sort=False)


In [5]:
def clean_texts(text, pattern):
    process = re.findall(pattern, text)
    for i in process:
        text = re.sub(i, '', text) 
    return text

In [6]:
# Clean text 

# Add new column 'tidy_text'
# Clean usernames
allTexts['tidy_text'] = np.vectorize(clean_texts)(
    allTexts['text'], "@[\w]*") 

# Clean non-alphabets
allTexts['tidy_text'] = allTexts['tidy_text'].str.replace(
    "[^a-zA-Z]", " ") 

allTexts.tail()

/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_11884/4189919608.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  allTexts['tidy_text'] = allTexts['tidy_text'].str.replace(


,id,label,text,tidy_text
32034,32035,NaN,{'Hatewatch'},Hatewatch
32035,32036,NaN,{'Intelligence Report'},Intelligence Report
32036,32037,NaN,{'Publications'},Publications
32037,32038,NaN,{'Law Enforcement'},Law Enforcement
32038,32039,NaN,{'Activist Toolkits'},Activist Toolkits


In [7]:
# Tokenization
tokenized_text = allTexts['tidy_text'].apply(lambda x: x.split())

tokenized_text.tail()

32034               [Hatewatch]
32035    [Intelligence, Report]
32036            [Publications]
32037        [Law, Enforcement]
32038      [Activist, Toolkits]
Name: tidy_text, dtype: object

In [8]:
tokenized_text = tokenized_text.apply(
    lambda x: [lemmatizer.lemmatize(i) for i in x])  

# Combine tokens back together
for i in range(len(tokenized_text)):
    tokenized_text[i] = ' '.join(tokenized_text[i])

# Replace with lemmatized
allTexts['tidy_text'] = tokenized_text




In [9]:
# Bag Of Words Model
bow_vectorizer = CountVectorizer(
    max_df=0.90, 
    min_df=2, 
    max_features=6000, 
    stop_words='english'
    )

BoW

In [10]:
# Feature extraction - Bag-of-Words [sklearn’s CountVectorizer] 
# Matrix dimensions change accordingly to test data size

# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(allTexts['tidy_text'])
# bow.get_shape()

In [11]:
# Splitting feature matrix into training and testing matrices
train_bow = bow[:31962, :]
test_bow = bow[31962:, :]

# splitting data into training and validation set
# xtrain_bow training dataset
# xvalid_bow validation for training
# ytrain label vector
# yvalid validation label vector
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(train_bow, train['label'], test_size=0.1)

# train_bow feature matrix
# train['label'] label vector
# random_state shuffles data before split into training and testing
# test_size percentage of data gets tested on (0.9 training)


In [12]:
# BoW
logReg.fit(xtrain_bow, ytrain)                     

# predicting on the validation set
pred = logReg.predict_proba(xvalid_bow)
# if prediction is greater than or equal to 0.31 than 1 else 0
predNum = pred[:, 1] >= 0.31
predNum = predNum.astype(np.int)

f1_score(yvalid, predNum)  # calculating f1 score    

/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_11884/2532116876.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predNum = predNum.astype(np.int)


0.6914153132250581

In [13]:
# Predict test data using the bow model and Logistic Regression

test_pred = logReg.predict_proba(test_bow)
test_pred_int = test_pred[:, 1] >= 0.31
test_pred_int = test_pred_int.astype(np.int)
test['label'] = test_pred_int
submission = test[['id', 'label']]
# writing data to a CSV file
submission.to_csv('sub_lreg_bow.csv', index=False)

/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_11884/3719177678.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_pred_int = test_pred_int.astype(np.int)


In [14]:
svm = SVC(kernel='rbf', random_state=0, gamma=0.14, C=11)

svm.fit(xtrain_bow, ytrain)

prediction = svm.predict(xvalid_bow)
prediction_int = prediction.astype(np.int)

f1_score(yvalid, prediction_int)

/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_11884/470810405.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  prediction_int = prediction.astype(np.int)


0.676923076923077

In [15]:
# prediction on test set
test_pred = svm.predict(test_bow)

test_pred_int = test_pred.astype(np.int)
test['label'] = test_pred_int

submission = test[['id', 'label']]
# writing data to a CSV file
submission.to_csv('svmrbfbow.csv', index=False)

print("End SVM")

End SVM


/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_11884/352891496.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_pred_int = test_pred.astype(np.int)


In [16]:
# TF-IDF freq
tfidf_vectorizer = TfidfVectorizer(
    max_df=0.90, 
    min_df=2, 
    max_features=6000, 
    stop_words='english'
    )

In [17]:

# TF-IDF Features - Looks at frequency of occurence for terms/importance of the term

# TF = (Number of times term t appears in a document)/(Number of terms in the document)
# IDF = log(N/n), where, N is the number of documents and n is the number of documents a term t has appeared in.
# 
# TF-IDF = TF*IDF

# TF-IDF feature matrix
tfidf = tfidf_vectorizer.fit_transform(allTexts['tidy_text'])
# tfidf.get_shape()
# print(tfidf)

In [18]:
# tfidf feature matrix
train_tfidf = tfidf[:31962, :]
test_tfidf = tfidf[31962:, :]


# ytrain.index index of axis labels 
# extract label vectors from the feature matrix via matching index
xtrain_tfidf = train_tfidf[ytrain.index]
xvalid_tfidf = train_tfidf[yvalid.index]



In [19]:
logReg.fit(xtrain_tfidf, ytrain)                   # Train model

prediction = logReg.predict_proba(xvalid_tfidf)    # Predict validation set
prediction_int = prediction[:, 1] >= 0.2
prediction_int = prediction_int.astype(np.int)

f1_score(yvalid, prediction_int)


/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_11884/1561877750.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  prediction_int = prediction_int.astype(np.int)


0.6986899563318778

In [20]:
# Predict Test data using TD-IDF - Logistic regression

test_pred = logReg.predict_proba(test_tfidf)
test_pred_int = test_pred[:, 1] >= 0.3
test_pred_int = test_pred_int.astype(np.int)
test['label'] = test_pred_int
submission = test[['id', 'label']]
# writing data to a CSV file
submission.to_csv('sub_lreg_td-idf.csv', index=False)

# if '1' in svm.predict(test_tfidf):
#     print("it is racist")

/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_11884/1418068712.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_pred_int = test_pred_int.astype(np.int)


In [21]:
# Model using SVM
# ------------------------------------------------------------------------------------------------------------------

# svm.fit(xtrain_bow, ytrain)        # Build using bow

# svm.fit(xtrain_tfidf, ytrain)      # Build using TD-IDF

# SVC()

# submission = test[['id', 'label']]
# # writing data to a CSV file
# submission.to_csv('svmrbfbow.csv', index=False)

# print("End SVM")

In [22]:

svm = SVC(kernel='rbf', random_state=0, gamma=0.14, C=11)

svm.fit(xtrain_tfidf, ytrain) 

prediction = svm.predict(xvalid_tfidf)
prediction_int = prediction.astype(np.int)

f1_score(yvalid, prediction_int)

/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_11884/2208432941.py:6: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  prediction_int = prediction.astype(np.int)


0.7076167076167076

In [23]:
# prediction on test set
test_pred = svm.predict(test_tfidf)
test_pred_int = test_pred.astype(np.int)
test['label'] = test_pred_int

print(test_pred_int)

[0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 1 0 1 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1]


/var/folders/y0/sry46qbs59n25vtlyr3vfqrh0000gn/T/ipykernel_11884/173282858.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  test_pred_int = test_pred.astype(np.int)
